In [1]:
import numpy as np
import tensorflow as tf
import keras

Using TensorFlow backend.


## 1. Define Modified VGG-16 Architecture for MNIST

In [2]:
class VGG16:
    def __init__(self, img_size, num_categories, learning_rate):
        self._model = keras.models.Sequential()
        self._add_conv_2d_filter(64, input_shape=img_size, duplicate=2)
        self._add_max_pooling_layer()
        self._add_conv_2d_filter(128, duplicate=2)
        self._add_max_pooling_layer()
        self._add_conv_2d_filter(256, duplicate=3)
        self._add_max_pooling_layer()
#         self._add_conv_2d_filter(512, duplicate=3)
#         self._add_max_pooling_layer()
#         self._add_conv_2d_filter(512, duplicate=3)
#         self._add_max_pooling_layer()
        self._model.add(keras.layers.Flatten())
        self._add_dense_layer(2048, 'relu')
        self._add_dense_layer(2048, 'relu')
        self._add_dense_layer(num_categories, 'softmax')
        self._optimizer = keras.optimizers.Adam(lr=learning_rate)
        self._model.compile(
            loss='categorical_crossentropy',
            optimizer=self._optimizer,
            metrics=['accuracy']
        )
        
    def train(self, x, y, batch_size, epochs, dev_ratio=0.2):
        self._model.fit(
            x=x, y=y, batch_size=batch_size, epochs=epochs, validation_split=dev_ratio, verbose=1
        )
    
    def predict(self, x):
        return np.argmax(self._model.predict(x), axis=-1)
    
    def print_model_summary(self):
        self._model.summary()

    def _add_conv_2d_filter(self, filters, input_shape=None, duplicate=1):
        for idx in range(duplicate):
            kwargs = {
                'filters': filters,
                'kernel_size': 3,
                'strides': 1,
                'padding': 'same',
                'data_format': 'channels_last',
                'activation': 'relu',
                'kernel_initializer': 'glorot_normal'
            }
            if input_shape is not None and idx == 0:
                kwargs['input_shape'] = input_shape
            self._model.add(keras.layers.Conv2D(**kwargs))
    
    def _add_max_pooling_layer(self):
        self._model.add(keras.layers.MaxPool2D(pool_size=2, strides=2))
    
    def _add_dense_layer(self, units, activation):
        self._model.add(keras.layers.Dense(units=units, activation=activation, kernel_initializer='glorot_normal'))

## 2. Get MNIST Dataset

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
mnist = input_data.read_data_sets("data/mnist", one_hot=True, reshape=False)

Extracting data/mnist\train-images-idx3-ubyte.gz
Extracting data/mnist\train-labels-idx1-ubyte.gz
Extracting data/mnist\t10k-images-idx3-ubyte.gz
Extracting data/mnist\t10k-labels-idx1-ubyte.gz


In [6]:
model = VGG16(
    mnist.train.images.shape[1:],
    mnist.train.labels.shape[-1],
    1e-3
)

In [7]:
model.print_model_summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 64)        640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 128)       147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 256)         295168    
__________

In [8]:
model.train(
    x=mnist.train.images,
    y=mnist.train.labels,
    batch_size=512,
    epochs=100,
    dev_ratio=0.2
)

Train on 44000 samples, validate on 11000 samples
Epoch 1/100
44000/44000 [==============================] - 9s 207us/step - loss: 0.4833 - acc: 0.8423 - val_loss: 0.0711 - val_acc: 0.9779
Epoch 2/100
44000/44000 [==============================] - 6s 131us/step - loss: 0.0605 - acc: 0.9810 - val_loss: 0.0591 - val_acc: 0.9810
Epoch 3/100
44000/44000 [==============================] - 6s 132us/step - loss: 0.0367 - acc: 0.9880 - val_loss: 0.0359 - val_acc: 0.9894
Epoch 4/100
44000/44000 [==============================] - 6s 135us/step - loss: 0.0252 - acc: 0.9922 - val_loss: 0.0379 - val_acc: 0.9891
Epoch 5/100
44000/44000 [==============================] - 6s 135us/step - loss: 0.0235 - acc: 0.9927 - val_loss: 0.0413 - val_acc: 0.9892
Epoch 6/100
44000/44000 [==============================] - 6s 132us/step - loss: 0.0181 - acc: 0.9941 - val_loss: 0.0323 - val_acc: 0.9905
Epoch 7/100
44000/44000 [==============================] - 6s 132us/step - loss: 0.0141 - acc: 0.9954 - val_loss: 0.

44000/44000 [==============================] - 6s 129us/step - loss: 0.0040 - acc: 0.9990 - val_loss: 0.0414 - val_acc: 0.9925
Epoch 60/100
44000/44000 [==============================] - 6s 129us/step - loss: 0.0034 - acc: 0.9992 - val_loss: 0.0534 - val_acc: 0.9921
Epoch 61/100
44000/44000 [==============================] - 6s 129us/step - loss: 8.4731e-04 - acc: 0.9997 - val_loss: 0.0470 - val_acc: 0.9936
Epoch 62/100
44000/44000 [==============================] - 6s 129us/step - loss: 0.0067 - acc: 0.9984 - val_loss: 0.0488 - val_acc: 0.9905
Epoch 63/100
44000/44000 [==============================] - 6s 129us/step - loss: 0.0051 - acc: 0.9989 - val_loss: 0.0489 - val_acc: 0.9912
Epoch 64/100
44000/44000 [==============================] - 6s 129us/step - loss: 0.0056 - acc: 0.9987 - val_loss: 0.0392 - val_acc: 0.9937
Epoch 65/100
44000/44000 [==============================] - 6s 129us/step - loss: 0.0049 - acc: 0.9988 - val_loss: 0.0450 - val_acc: 0.9922
Epoch 66/100
44000/44000 [===

In [9]:
model.predict(mnist.test.images[:10])

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9], dtype=int64)

In [10]:
np.argmax(mnist.test.labels[:10], axis=-1)

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9], dtype=int64)

In [13]:
accuracy = np.count_nonzero(model.predict(mnist.test.images) == np.argmax(mnist.test.labels, axis=-1).ravel()) / mnist.test.labels.shape[0]

In [14]:
accuracy

0.9953